EfficientNetB7 for binary classification (COVID vs Normal) on a balanced custom lung X-ray dataset. This includes:

Train-validation-test split

Metrics plot (accuracy, precision, recall)

Confusion matrix visualization

Model summary (all layers printed for presentation)

Saving model and weights (for XAI like Grad-CAM)

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.callbacks import ModelCheckpoint

# Define path
path = "A:\\GVSU Semester 5\\PSM Internship\\Covid-19\\COVID-19_Radiography_Dataset"
max_images_per_class = 3616  # Balance classes

# Initialize containers
images = []
labels = []
IMG_SIZE = 224  # Resize to match EfficientNet input

# Load and preprocess images
for class_label in os.listdir(path):
    class_path = os.path.join(path, class_label)
    filenames = os.listdir(class_path)[:max_images_per_class]

    for fname in filenames:
        img_path = os.path.join(class_path, fname)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(class_label)

# Convert to NumPy and normalize
images = np.array(images, dtype='float32') / 255.0

# Encode labels to binary (Normal/COVID)
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = tf.keras.utils.to_categorical(labels)  # One-hot encoding

# Split dataset into train, val, test (80/10/10)
X_temp, test_x, y_temp, test_y = train_test_split(images, labels, test_size=0.1, random_state=42, stratify=labels)
train_x, val_x, train_y, val_y = train_test_split(X_temp, y_temp, test_size=0.1111, random_state=42, stratify=y_temp)

# Print dataset stats
print(f"Train shape: {train_x.shape}, {train_y.shape}")
print(f"Validation shape: {val_x.shape}, {val_y.shape}")
print(f"Test shape: {test_x.shape}, {test_y.shape}")

# Create tf.data pipelines for performance
BATCH_SIZE = 32
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(BATCH_SIZE)

# Build EfficientNetB7 model (no pretraining, end-to-end training)
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = EfficientNetB7(include_top=False, weights=None, input_tensor=inputs)
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(2, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Print model summary
model.summary()

# Save best model during training
checkpoint = ModelCheckpoint("efficientnetb7_covid_model.h5", monitor="val_accuracy", save_best_only=True, verbose=1)

# Train model
history = model.fit(train_dataset,
                    epochs=10,
                    validation_data=val_dataset,
                    callbacks=[checkpoint],
                    verbose=2)

# Evaluate on test set
test_loss, test_acc, test_precision, test_recall = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}")

# Save final model (for XAI/Grad-CAM etc.)
model.save("efficientnetb7_covid_model_final")

# Plot training history
def plot_training_metrics(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    prec = history.history['precision']
    val_prec = history.history['val_precision']
    recall = history.history['recall']
    val_recall = history.history['val_recall']
    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 3, 1)
    plt.plot(epochs_range, acc, label='Train Accuracy')
    plt.plot(epochs_range, val_acc, label='Val Accuracy')
    plt.title('Accuracy')
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.plot(epochs_range, prec, label='Train Precision')
    plt.plot(epochs_range, val_prec, label='Val Precision')
    plt.title('Precision')
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.plot(epochs_range, recall, label='Train Recall')
    plt.plot(epochs_range, val_recall, label='Val Recall')
    plt.title('Recall')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_training_metrics(history)

# Predict test set for confusion matrix
y_pred_probs = model.predict(test_x)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(test_y, axis=1)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=le.classes_))
